In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import numpy as np
import os
import random

# Configurações de semente para reprodutibilidade
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# Ativa mixed precision
mixed_precision.set_global_policy('mixed_float16')
tf.config.optimizer.set_jit(True)

print("GPU Disponível:", tf.config.list_physical_devices('GPU'))

GPU Disponível: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(8192)

In [ ]:
# Aplica transformações de aumento de dados
train_dataset = train_dataset.map(
    lambda image, label: (
        tf.image.random_flip_left_right(image),
        tf.cast(label, tf.int32)
    ),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_dataset = train_dataset.map(
    lambda image, label: (
        tf.image.random_brightness(image, 0.05),
        label
    ),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_dataset = train_dataset.map(
    lambda image, label: (
        tf.image.random_contrast(image, 0.95, 1.05),
        label
    ),
    num_parallel_calls=tf.data.AUTOTUNE
)

In [ ]:
# Aplica CutMix simplificado
batch_size = 128
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.map(
    lambda images, labels: (
        tf.cond(
            tf.random.uniform(()) > 0.1,
            lambda: (images, labels),
            lambda: (tf.identity(images), tf.identity(labels))
        )
    ),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.RandomRotation(0.01, fill_mode='nearest'),
    layers.RandomZoom(0.03, fill_mode='nearest'),
    layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Conv2D(64, (3, 3), padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Conv2D(128, (3, 3), padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Conv2D(256, (3, 3), padding='same', kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(512, kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Dropout(0.35),
    layers.Dense(256, kernel_initializer='he_uniform'),
    layers.BatchNormalization(),
    layers.Activation('gelu'),
    layers.Dropout(0.35),
    layers.Dense(10, dtype='float32')
])

opt = tf.keras.optimizers.AdamW(learning_rate=0.0015, weight_decay=1e-4)
model.compile(
    optimizer=opt,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5_acc')]
)

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=12, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
]

In [ ]:
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 43s 69ms/step - accuracy: 0.3773 - loss: 1.7447 - top5_acc: 0.8580 - val_accuracy: 0.6195 - val_loss: 1.0885 - val_top5_acc: 0.9640 - learning_rate: 0.0015
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 33s 66ms/step - accuracy: 0.6325 - loss: 1.0319 - top5_acc: 0.9651 - val_accuracy: 0.6654 - val_loss: 1.0060 - val_top5_acc: 0.9694 - learning_rate: 0.0015
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.7093 - loss: 0.8237 - top5_acc: 0.9787 - val_accuracy: 0.7480 - val_loss: 0.7366 - val_top5_acc: 0.9821 - learning_rate: 0.0015
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.7509 - loss: 0.7188 - top5_acc: 0.9846 - val_accuracy: 0.7715 - val_loss: 0.6572 - val_top5_acc: 0.9855 - learning_rate: 0.0015
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.7779 - loss: 0.6400 - top5_acc: 0.9872 - val_accuracy: 0.7614 - val_loss: 0.7100 - val_top5_acc: 0.9883 - learning_rate: 0.0015
Epoch 6/100
391

In [ ]:
# Avalia o modelo
test_loss, test_acc, test_top5_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test top-5 accuracy: {test_top5_acc:.4f}")

# Calcula a média (accuracy + val_accuracy) / 2 para a última época
mean_acc_last_epoch = (history.history['accuracy'][-1] + history.history['val_accuracy'][-1]) / 2
print(f"Mean accuracy for last epoch: {mean_acc_last_epoch:.4f}")

# Visualiza os resultados
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treinamento')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend(loc='lower right')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treinamento')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend(loc='upper right')
plt.grid(True)

plt.tight_layout()
plt.savefig('training_metrics.png')
plt.show()